In [8]:
import pandas as pd
import talib
from talib.abstract import *


In [7]:
#installation worked only after installing the dependancy packages using (brew install ta-lib)
!pip install TA-Lib


  Using cached TA_Lib-0.4.32-cp311-cp311-macosx_14_0_arm64.whl


In [14]:
pwd

'/Users/abhisheksuga/Documents/Edu/thesis/gold/src'

In [20]:
df =  pd.read_csv("../data/gold.csv",header=0, index_col=0, parse_dates=True)

In [10]:
df.head()

,Open,High,Low,Close,Volume,Currency
Date,,,,,,
2000-01-04,289.5,289.5,280.0,283.7,21621,USD
2000-01-05,283.7,285.0,281.0,282.1,25448,USD
2000-01-06,281.6,282.8,280.2,282.4,19055,USD
2000-01-07,282.5,284.5,282.0,282.9,11266,USD
2000-01-10,282.4,283.9,281.8,282.7,30603,USD


In [11]:
len(df)

5703

In [12]:
df.isna().sum()

Open        0
High        0
Low         0
Close       0
Volume      0
Currency    0
dtype: int64

In [13]:
df = df[df.Open !=0]
df = df[df.High !=0]
df = df[df.Low !=0]
df = df[df.Close !=0]

In [14]:
def Add_Lag(data,col,lag_list):
    
    position = data.columns.tolist().index(col)
    
    for lag in lag_list:
    
        col_val = data[col].shift(lag)

        col_name = col+'_lag:'+'{length}'.format(length=lag)

        data.insert(loc=position, column=col_name, value=col_val)
        

    return data

In [22]:
def Indicator(Data):

    data = pd.DataFrame()
    data['Open'] = Data['Open']
    data['High'] = Data['High']
    data['Low'] = Data['Low']
    data['Close'] = Data['Close']
    data['Volume'] = Data['Volume']


    Orginal_columns_names = ['Open', 'High', 'Low', 'Close', 'Volume']
    
    #we apply indicators on one lage data
    Open = data['Open'].shift(1)
    High = data['High'].shift(1)
    Low = data['Low'].shift(1)
    Close = data['Close'].shift(1)
    Volume = data['Volume'].shift(1)

    Orginal_columns_values = {'open':Open,'high':High,'low':Low,'close':Close,'volume':Volume}
    #Gathering all functions in the Talib library
    Talib = talib.get_function_groups()
    Groups = list(Talib.keys())
    Indicators = []
    for group in Groups:
        for g in Talib[group]:
            Indicators.append(g)
    #As the following two functions are not returning an array with length of our data.shapep[0], we remove it
    Indicators.remove('MAVP')
    Indicators.remove('OBV')

    for indicator in Indicators:
        indicator = eval(indicator)
        
        #Here we have two types of functions(indicators,...):
        #The ones need just one column and the others which need more
        if len(indicator.output_names)==1:

            if list(indicator.get_input_names().values()) == ['close']:

                for col in Orginal_columns_names:

                    lagged_col = data[col].shift(1)

                    new_col = indicator(lagged_col)

                    name = indicator.info['name']

                    col_name = col+'_'+'{Name}'.format(Name=name)

                    data[col_name] = new_col

            
            else: #Functions need more than one features, e.g. [High, Low, Close, Open]

                name = indicator.info['name']

                inputs = list(indicator.get_input_names().values())

                if len(inputs)==2:

                    new_col = indicator(Orginal_columns_values[inputs[0]],Orginal_columns_values[inputs[1]])

                    data[name] = new_col
                    
                elif len(inputs)==3:

                    new_col = indicator(Orginal_columns_values[inputs[0]],Orginal_columns_values[inputs[1]],Orginal_columns_values[inputs[2]])
                    
                    data[name] = new_col

                elif len(inputs)==4:
                    
                    new_col = indicator(Orginal_columns_values[inputs[0]],Orginal_columns_values[inputs[1]],Orginal_columns_values[inputs[2]],Orginal_columns_values[inputs[3]])
                    
                    data[name] = new_col
    
    remove_col = []
    for col in data.columns:
        if float('-inf') in data[col].values: #Remove columns that have very large values
            remove_col.append(col)
        if data.tail(data.shape[0]-100)[col].isnull().values.any()==True: #remove columns that have nan values which are not cause of lags
            remove_col.append(col)
            
    remove_col = remove_col+Orginal_columns_names
    data.drop(remove_col,axis=1,inplace=True)

    return data



In [23]:
new_data = Indicator(df)

/var/folders/bz/q_7jbqjx7hj_vffhkj3cyl_m0000gn/T/ipykernel_11060/2626521298.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[col_name] = new_col
/var/folders/bz/q_7jbqjx7hj_vffhkj3cyl_m0000gn/T/ipykernel_11060/2626521298.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[col_name] = new_col
/var/folders/bz/q_7jbqjx7hj_vffhkj3cyl_m0000gn/T/ipykernel_11060/2626521298.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

In [25]:
new_data.tail()

,Open_HT_DCPERIOD,High_HT_DCPERIOD,Low_HT_DCPERIOD,Close_HT_DCPERIOD,Volume_HT_DCPERIOD,Open_HT_DCPHASE,High_HT_DCPHASE,Low_HT_DCPHASE,Close_HT_DCPHASE,Volume_HT_DCPHASE,...,Open_TSF,High_TSF,Low_TSF,Close_TSF,Volume_TSF,Open_VAR,High_VAR,Low_VAR,Close_VAR,Volume_VAR
Date,,,,,,,,,,,,,,,,,,,,,
2022-08-29,31.752941,32.030096,32.659662,31.287085,16.152469,218.007277,223.916304,217.119858,230.966272,104.518981,...,1750.074725,1756.134066,1736.056044,1741.780220,141213.351648,49.5056,30.3264,69.7344,72.0784,5.289194e+08
2022-08-30,32.011496,31.958492,32.307232,30.931944,16.730867,227.480301,233.679087,231.077481,240.603608,111.540984,...,1743.103297,1750.427473,1728.779121,1738.156044,146269.263736,78.2976,46.4880,114.7200,67.1176,5.778547e+08
2022-08-31,31.730026,31.443452,31.601386,30.149053,17.143039,237.078920,248.832923,240.746986,256.573695,136.209755,...,1739.469231,1746.548352,1724.858242,1733.013187,148838.648352,79.0384,91.0584,150.1344,141.6184,5.912725e+08
2022-09-01,30.928059,30.557495,30.625783,29.161273,17.379605,252.491099,259.831106,256.723250,275.006681,147.799296,...,1733.150549,1739.102198,1718.917582,1726.393407,159498.120879,163.2496,208.9944,212.2416,231.4216,5.779119e+08
2022-09-02,29.851517,29.530152,29.592532,28.256961,17.630448,269.927018,278.040139,274.686404,295.796639,146.948341,...,1725.172527,1730.643956,1709.581319,1719.074725,174123.373626,264.9280,286.6680,247.5144,234.4024,5.251140e+08
